# TME 5 TAL


In [41]:
import numpy as  np
import codecs
import matplotlib.pyplot as plt
import unicodedata
import re
from collections import Counter,defaultdict
import nltk.corpus.reader as pt
import os

import string
import pdb
import nltk
import scipy

## 1. Chargement des données (automatique)



In [2]:
import nltk.corpus.reader as pt
path2data = u'20news-bydate/20news-bydate-train/'

rdr = pt.CategorizedPlaintextCorpusReader(path2data, '.*/[0-9]+', encoding='latin1', cat_pattern='([\w\.]+)/*')
docs = [[rdr.raw(fileids=[f]) for f in rdr.fileids(c) ] for c in rdr.categories()]

## 2. Chargement des données (à la main)

In [3]:
def readAFile(nf):
    f = open(nf, 'rb')

    txt = f.readlines()
    txt = ' '.join(txt)

    f.close()
    return txt

def compteLignes(nf, fdl='\n', tbuf=16384):
    """Compte le nombre de lignes du fichier nf"""
    c = 0
    f = open(nf, 'rb')
    while True:
        buf = None
        buf = f.read(tbuf)
        if len(buf)==0:
            break
        c += buf.count(fdl)
    f.seek(-1, 2)
    car = f.read(1)
    if car != fdl:
        c += 1
    f.close()
    return c

## sac de mots

In [4]:
def preprocessSent(s):
    punc = u''.join([ch for ch in string.punctuation]) # je laisse les points pour pouvoir séparer les phrases
    punc += u'\n\r\t\\'
    table =string.maketrans(punc + string.digits, ' '*(len(punc)+len(string.digits)))
    s = string.translate(unicodedata.normalize("NFKD",s).encode("ascii","ignore"),table).lower() # elimination des accents + minuscules
    return s
def count_ngrams(s,n=2,dic=None):
    if dic is None:
        dic=Counter()
    for i in range(len(s)):
        for j in range(i+1,min(i+n+1,len(s)+1)):
            dic[u''.join(s[i:j])]+=1
    return dic

In [5]:
docProPre = [[] for i in range(len(docs))]
for i in range(len(docs)):
    for j in range(len(docs[i])):
        docProPre[i].append(preprocessSent(docs[i][j]))
    docProPre[i]=[s[:-1] for s in docProPre[i] if len(s)>2] # elimination des phrases/docs vides

In [6]:
docProPre[0][0]


'from  mathew  mathew mantis co uk  subject  alt atheism faq  atheist resources summary  books  addresses  music    anything related to atheism keywords  faq  atheism  books  music  fiction  addresses  contacts expires  thu     apr               gmt distribution  world organization  mantis consultants  cambridge  uk  supersedes                  mantis co uk  lines       archive name  atheism resources alt atheism archive name  resources last modified     december      version                                     atheist resources                        addresses of atheist organizations                                       usa  freedom from religion foundation  darwin fish bumper stickers and assorted other atheist paraphernalia are available from the freedom from religion foundation in the us   write to   ffrf  p o  box      madison  wi        telephone                  evolution designs  evolution designs sell the  darwin fish    it s a fish symbol  like the ones christians stick on 

In [7]:
indToDic = {}
dicToInd = {}
v = np.unique(np.array([a for doc in docProPre for w in doc for a in w.split()]))
for i,w in enumerate(v):
    
    dicToInd[i]=w
    indToDic[w]=i

In [8]:
D = len(dicToInd)
docP = [docProPre[i][j].split() for i in range(len(docProPre)) for j in range(len(docProPre[i]))  ]
N=len(docP)



In [9]:
bow=scipy.sparse.lil_matrix((N,D))
cpt=0
def add1(i,j):
    bow[i,j] += 1

In [10]:

[[add1(i,indToDic[m]) for m in s if m in indToDic] for i,s in enumerate(docP)]

print "bow computed"

bow computed


In [11]:
bowN = bow / bow.sum(1)

In [116]:
import numpy.random as rand
prototypes = bowN[rand.permutation(N)[:20]]

In [57]:
sim = bowN.dot(prototypes.T) # métrique type cos
y= sim.argmax(1) # comprendre les dimensions des matrices pour comprendre ce calcul


In [117]:
for ite in range(20):
    sim = bowN.dot(prototypes.T) # métrique type cos
    y= sim.argmax(1) # comprendre les dimensions des matrices pour comprendre ce calcul
    for i in np.unique(np.array(y)):
        prototypes[i] = bowN[np.where(y==i)[0]].sum(axis=1)/np.array(np.where(y==i)).shape[2]
    print 'itérations '+str(ite) +" check"

itérations 0 check
itérations 1 check
itérations 2 check
itérations 3 check
itérations 4 check
itérations 5 check
itérations 6 check
itérations 7 check
itérations 8 check
itérations 9 check
itérations 10 check
itérations 11 check
itérations 12 check
itérations 13 check
itérations 14 check
itérations 15 check
itérations 16 check
itérations 17 check
itérations 18 check
itérations 19 check


In [138]:
ySur = [i for i,doc in enumerate(docs) for di in doc]


In [161]:
purete = np.zeros((20))
for i in range(20):
    purete[i] = np.where(y==i)==np.where(ySur==i)
    

TypeError: only length-1 arrays can be converted to Python scalars

In [123]:
np.unique(np.array(y))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19], dtype=int64)

In [122]:
save = y

In [128]:
purete = np.zeros(y.shape[0])

In [146]:
print ySur[np.where(ySur==1)[0]]


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [170]:
purete = np.zeros((20,20))
purete

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  

In [221]:
nb_classes=20
purete = np.zeros((20,2))

for c in range(nb_classes):
    index=np.where(y==c)[0]
    #print ySur[index]
    salut=np.bincount(ySur[index][0])
    m=np.max(salut)*100./np.array(np.where(y==c)).shape[2]
    
    
    
    
    purete[c] = np.argmax(salut),m
    
    #     print np.argmax([[1. for cl in range(20)] for i in range(len(ySur)) if(ySur==cl)])
    #for j in range(20):
        
    
print purete


[[  9.          42.22222222]
 [  0.          15.82278481]
 [ 10.          90.47619048]
 [  6.          66.66666667]
 [  9.          12.92517007]
 [  6.          51.18110236]
 [  5.          29.06976744]
 [  5.          34.48275862]
 [  1.          14.28571429]
 [ 12.          12.05479452]
 [  9.          15.29680365]
 [  8.          20.        ]
 [  1.          16.52173913]
 [ 10.          16.        ]
 [ 11.           6.89655172]
 [ 12.          12.25806452]
 [  6.          50.        ]
 [  2.          12.68715524]
 [  5.          24.        ]
 [  7.          10.6918239 ]]


In [216]:
index = np.where(y==0)[0]
print np.bincount(ySur[index][0])*1./np.array(np.where(y==0)).shape[2]

[ 0.02222222  0.          0.          0.          0.          0.
  0.02222222  0.02222222  0.02222222  0.42222222  0.13333333  0.          0.
  0.          0.04444444  0.15555556  0.04444444  0.          0.02222222
  0.08888889]


In [225]:
index = np.where(y==0)

In [233]:
i = (index[0])

In [238]:
i[0][0][0][0][0]

matrix([[  402,  3700,  4126,  4668,  5197,  5213,  5214,  5245,  5364,
          5398,  5404,  5445,  5446,  5494,  5599,  5612,  5618,  5634,
          5643,  5717,  5748,  5762,  5776,  5793,  5975,  6252,  6257,
          6260,  6359,  8558,  8704,  8922,  9000,  9005,  9025,  9064,
          9204,  9230,  9568,  9599, 10535, 11137, 11165, 11211, 11232]], dtype=int64)